## Fire Detection TinyML Model


In [1]:
import tensorflow as tf
import scipy

import tensorflow.keras as keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Activation

from tensorflow.keras.preprocessing.image import ImageDataGenerator


2022-10-05 00:32:58.021404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 00:33:07.713264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-05 00:33:07.713324: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-05 00:33:09.209540: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-05 00:33:21.853913: W tensorflow/stream_executor/platform/de

In [2]:
model = keras.Sequential()


model.add(Conv2D(12, kernel_size=3, strides=2,input_shape=(160,120,3), kernel_constraint=tf.keras.constraints.MaxNorm(1),activation='relu', padding='same'))

model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))

model.add(Conv2D(36, kernel_size=3, strides=2,activation='relu', padding='same'))

model.add(Dropout(0.2))

model.add(Conv2D(72, kernel_size=3, strides=2, activation='relu', padding='same'))

model.add(Dropout(0.2))
model.add(GlobalMaxPooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))


2022-10-04 10:49:49.554175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 10:49:49.554673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-04 10:49:49.554760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-04 10:49:49.554820: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-04 10:49:49.554874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# Data Preprocessing


In [4]:
#collected dataset of around 1200 photos

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True, vertical_flip =True,brightness_range=[0.8,1.2],zoom_range=[0.8,1.2])
test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
        '/home/gabrielmadruga/Documents/Github/tcc-wildfire/dataset/Test',
        target_size=(160, 120),
        color_mode='rgb',
        batch_size=64,
         class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
        '/home/gabrielmadruga/Documents/Github/tcc-wildfire/dataset/Training',
        target_size=(160, 120),
        batch_size=64,
        color_mode='rgb',
        class_mode='categorical')

Found 8617 images belonging to 2 classes.
Found 39375 images belonging to 2 classes.


# Training


In [5]:
history = model.fit(
        train_generator,
        epochs=5,
        validation_data=validation_generator,
      verbose=1)

2022-10-04 10:50:07.674238: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600 exceeds 10% of free system memory.


Epoch 1/5


2022-10-04 10:50:11.272554: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600 exceeds 10% of free system memory.
2022-10-04 10:50:12.423217: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14826240 exceeds 10% of free system memory.
2022-10-04 10:50:12.470197: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 21772800 exceeds 10% of free system memory.
2022-10-04 10:50:12.472791: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 21772800 exceeds 10% of free system memory.


616/616 [==============================] - 1745s 3s/step - loss: 0.1873 - accuracy: 0.9350 - val_loss: 1.1238 - val_accuracy: 0.5805
Epoch 2/5
616/616 [==============================] - 1365s 2s/step - loss: 0.1295 - accuracy: 0.9595 - val_loss: 1.1253 - val_accuracy: 0.5808
Epoch 3/5
616/616 [==============================] - 1312s 2s/step - loss: 0.1116 - accuracy: 0.9632 - val_loss: 0.8720 - val_accuracy: 0.6120
Epoch 4/5
616/616 [==============================] - 1525s 2s/step - loss: 0.0946 - accuracy: 0.9683 - val_loss: 1.2881 - val_accuracy: 0.5923
Epoch 5/5
616/616 [==============================] - 540s 878ms/step - loss: 0.0805 - accuracy: 0.9729 - val_loss: 1.3260 - val_accuracy: 0.6168


In [6]:
#save the model to use later
model.save("/home/gabrielmadruga/Documents/Github/tcc-wildfire/models") 


INFO:tensorflow:Assets written to: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models/assets


INFO:tensorflow:Assets written to: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models/assets


# Quantizing The Model


In [4]:
import glob
import os
import pandas as pd
import numpy as np

In [5]:
converter = tf.lite.TFLiteConverter.from_saved_model("/home/gabrielmadruga/Documents/Github/tcc-wildfire/models")
model_no_quant_tflite = converter.convert()


2022-10-05 01:18:23.255568: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-05 01:18:23.255600: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-05 01:18:23.255745: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:23.258183: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-05 01:18:23.258200: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:23.265876: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-05 01:18:23.317414: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:

In [6]:
# converter = tf.lite.TFLiteConverter.from_saved_model("/home/gabrielmadruga/Documents/Github/tcc-wildfire/models")
# model_no_quant_tflite = converter.convert()

# Save the model to disk
open("float.tflite", "wb").write(model_no_quant_tflite)
def representative_dataset():
  for filename in glob.glob("/home/gabrielmadruga/Documents/Github/tcc-wildfire/dataset/Test" + "/*/*.jpg"):
    img = keras.preprocessing.image.load_img(filename, target_size=(160, 120))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis for images, labels in train_ds.take(1):
    yield([img_array])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open("quantized.tflite", "wb").write(model_tflite)

2022-10-05 01:18:23.489774: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-10-05 01:18:23.489804: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-10-05 01:18:23.489944: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:23.491924: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-10-05 01:18:23.491948: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:23.499698: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-10-05 01:18:23.555424: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /home/gabrielmadruga/Documents/Github/tcc-wildfire/models
2022-10-05 01:18:

42824

##Comparing Sizes Between Models


In [7]:
def get_dir_size(dir):
  size = 0
  for f in os.scandir(dir):
    if f.is_file():
      size += f.stat().st_size
    elif f.is_dir():
      size += get_dir_size(f.path)
  return size

# Calculate size
size_tf = get_dir_size("/home/gabrielmadruga/Documents/Github/tcc-wildfire/models")
size_no_quant_tflite = os.path.getsize("float.tflite")
size_tflite = os.path.getsize("quantized.tflite")

# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

,Size,
Model,,
TensorFlow,668385 bytes,
TensorFlow Lite,142044 bytes,(reduced by 526341 bytes)
TensorFlow Lite Quantized,42824 bytes,(reduced by 99220 bytes)


##Testing It Out

In [8]:
def predict_tflite(tflite_model, filename):
  img = keras.preprocessing.image.load_img(filename, target_size=(160, 120))
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # If required, quantize the input layer (from float to integer)
  input_scale, input_zero_point = input_details["quantization"]
  if (input_scale, input_zero_point) != (0.0, 0):
    img_array = np.multiply(img_array, 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])
  
  # Invoke the interpreter
  interpreter.set_tensor(input_details["index"], img_array)
  interpreter.invoke()
  pred = interpreter.get_tensor(output_details["index"])[0]
  
  # If required, dequantized the output layer (from integer to float)
  output_scale, output_zero_point = output_details["quantization"]
  if (output_scale, output_zero_point) != (0.0, 0):
    pred = pred.astype(np.float32)
    pred = np.multiply((pred - output_zero_point), output_scale)
  
  predicted_label_index = np.argmax(pred)
  predicted_score = pred[predicted_label_index]
  return (predicted_label_index, predicted_score)

In [18]:
print(predict_tflite(model_tflite,"/home/gabrielmadruga/Documents/Github/tcc-wildfire/dataset/Images/image_145.jpg")) #fires are (0,1) and non fires are a (1,1)

(1, 0.99609375)


##Exporting the Quantized Model

In [ ]:
!apt-get update && apt-get -qq install xxd

In [ ]:
!xxd -i quantized.tflite > model_data.cc